In [5]:
states = ["California", "New York", "Florida", "Vermont", "California"]

In [7]:
states_new_list=[est for est in states if est == 'California' ]
states_new_list
#After this I will count

print(f'California quantity: {len(states_new_list)}')

California cuantity: 2


In [1]:
from pyspark import SparkContext
from pyspark.sql import SparkSession, SQLContext, functions as f
from pyspark.sql.types import StructField, StructType, IntegerType, StringType, FloatType
from IPython.display import display

In [2]:
spark=SparkSession.builder\
    .master('local')\
    .appName('test_session_01')\
    .getOrCreate()

23/11/14 08:27:08 WARN Utils: Your hostname, CompuDell01 resolves to a loopback address: 127.0.1.1; using 192.168.29.145 instead (on interface eth0)
23/11/14 08:27:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/11/14 08:27:10 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [14]:
!pwd
!ls -lac ./curso_spark_git_clone/curso-apache-spark-platzi/files

/home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/39_Spark_y_Big_Data_Fundamentos
total 14524
drwxr-xr-x 3 compu_dell_ubuntu_01 compu_dell_ubuntu_01    4096 Oct 26 12:13 .
drwxr-xr-x 5 compu_dell_ubuntu_01 compu_dell_ubuntu_01    4096 Oct  7 13:11 ..
-rw-r--r-- 1 compu_dell_ubuntu_01 compu_dell_ubuntu_01     946 Oct  7 13:11 deporte.csv
drwxr-xr-x 2 compu_dell_ubuntu_01 compu_dell_ubuntu_01    4096 Oct 26 12:13 deporte_partitions
-rw-r--r-- 1 compu_dell_ubuntu_01 compu_dell_ubuntu_01 2764536 Oct  7 13:11 deportista.csv
-rw-r--r-- 1 compu_dell_ubuntu_01 compu_dell_ubuntu_01 2807091 Oct  7 13:11 deportista2.csv
-rw-r--r-- 1 compu_dell_ubuntu_01 compu_dell_ubuntu_01 2776782 Oct  7 13:11 deportistaError.csv
-rw-r--r-- 1 compu_dell_ubuntu_01 compu_dell_ubuntu_01   32942 Oct 24 13:40 evento.csv
-rw-r--r-- 1 compu_dell_ubuntu_01 compu_dell_ubuntu_01    1978 Oct  7 13:11 juegos.csv
-rw-r--r-- 1 compu_dell_ubuntu_01 compu_dell_ubuntu_01  255853 Oct  7 13:11 modelo_relacional

In [3]:
general_path='./curso_spark_git_clone/curso-apache-spark-platzi/files/'

In [4]:
deporte_df=spark.read.options(inferSchema=True, header=True, delimiter=',')\
    .csv(general_path + 'deporte.csv')

deportista_df=spark.read.options(inferSchema=True, header=True, delimiter=',')\
    .csv(general_path+'deportista.csv')

deportista2_df=spark.read.options(inferSchema=True, header=False, delimiter=',')\
    .csv(general_path+'deportista2.csv')

evento_df=spark.read.options(inferSchema=True, header=True, delimiter=',')\
    .csv(general_path+'evento.csv')

juegos_df=spark.read.options(inferSchema=True, header=True, delimiter=',')\
    .csv(general_path+'juegos.csv')

paises_df=spark.read.options(inferSchema=True, header=True, delimiter=',')\
    .csv(general_path+'paises.csv')

resultados_df=spark.read.options(inferSchema=True, header=True, delimiter=',')\
    .csv(general_path+'resultados.csv')

print('deporte_df')
deporte_df.printSchema()
print('deportista_df')
deportista_df.printSchema()
print('deportista2_df')
deportista2_df.printSchema()
print('evento_df')
evento_df.printSchema()
print('juegos_df')
juegos_df.printSchema()
print('paises_df')
paises_df.printSchema()
print('resultados_df')
resultados_df.printSchema()

deporte_df
root
 |-- deporte_id: integer (nullable = true)
 |-- deporte: string (nullable = true)

deportista_df
root
 |-- deportista_id: integer (nullable = true)
 |-- nombre: string (nullable = true)
 |-- genero: integer (nullable = true)
 |-- edad: integer (nullable = true)
 |-- altura: integer (nullable = true)
 |-- peso: double (nullable = true)
 |-- equipo_id: integer (nullable = true)

deportista2_df
root
 |-- _c0: integer (nullable = true)
 |-- _c1: string (nullable = true)
 |-- _c2: integer (nullable = true)
 |-- _c3: integer (nullable = true)
 |-- _c4: integer (nullable = true)
 |-- _c5: double (nullable = true)
 |-- _c6: integer (nullable = true)

evento_df
root
 |-- evento_id: integer (nullable = true)
 |-- evento: string (nullable = true)
 |-- deporte_id: integer (nullable = true)

juegos_df
root
 |-- _c0: integer (nullable = true)
 |-- nombre_juego: string (nullable = true)
 |-- annio: integer (nullable = true)
 |-- temporada: string (nullable = true)
 |-- ciudad: string 

In [5]:
new_deportista_df = deportista_df.union(deportista2_df)
new_deportista_df.filter(new_deportista_df.deportista_id=='_c0').show()

print(new_deportista_df.count() == deportista_df.count()+deportista2_df.count())


+-------------+------+------+----+------+----+---------+
|deportista_id|nombre|genero|edad|altura|peso|equipo_id|
+-------------+------+------+----+------+----+---------+
+-------------+------+------+----+------+----+---------+

True


In [6]:
dep_team = new_deportista_df.join(paises_df, new_deportista_df.equipo_id == paises_df.id, 'left')
dep_team = dep_team.select('deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo')
dep_team = dep_team.withColumn('genero', f.when(dep_team.genero==1, 'male').otherwise('female'))
dep_team.orderBy(f.desc('altura')).show(15)

+-------------+--------------------+------+----+------+-----+-------------+
|deportista_id|              nombre|genero|edad|altura| peso|       equipo|
+-------------+--------------------+------+----+------+-----+-------------+
|       132627|            Yao Ming|  male|  20|   226|141.0|        China|
|        16639|Tommy Loren Burleson|  male|  20|   223|102.0|United States|
|       104059|Arvydas Romas Sab...|  male|  23|   223|122.0| Soviet Union|
|         9407|      Gunther Behnke|  male|  29|   221|114.0|      Germany|
|       134621|        Zhang Zhaoxu|  male|  24|   221|110.0|        China|
|        30409|Roberto Dueas Her...|  male|  24|   221|137.0|        Spain|
|        71228|Lucien James Luc ...|  male|  19|   220|135.0|    Australia|
|        91087|Viktor Aleksandro...|  male|  31|   220|112.0| Soviet Union|
|       120589|Volodymyr Petrovy...|  male|  18|   220|110.0| Soviet Union|
|        83657|Dmitry Aleksandro...|  male|  23|   219|104.0|       Russia|
|        987

In [7]:
resultados_df = resultados_df.withColumnRenamed("deportista_id", "dep_id")
resultados_df.show(5)

+------------+-------+------+--------+---------+
|resultado_id|medalla|dep_id|juego_id|evento_id|
+------------+-------+------+--------+---------+
|           1|     NA|     1|      39|        1|
|           2|     NA|     2|      49|        2|
|           3|     NA|     3|       7|        3|
|           4|   Gold|     4|       2|        4|
|           5|     NA|     5|      36|        5|
+------------+-------+------+--------+---------+
only showing top 5 rows



In [8]:
dep_team_year = dep_team.join(resultados_df, dep_team.deportista_id==resultados_df.dep_id, 'left')
dep_team_year.show(5)
dep_team_year = dep_team_year.select('deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo', 'medalla', 'juego_id')
dep_team_year.show(15)

+-------------+--------------------+------+----+------+----+--------------+------------+-------+------+--------+---------+
|deportista_id|              nombre|genero|edad|altura|peso|        equipo|resultado_id|medalla|dep_id|juego_id|evento_id|
+-------------+--------------------+------+----+------+----+--------------+------------+-------+------+--------+---------+
|            1|           A Dijiang|  male|  24|   180|80.0|         China|           1|     NA|     1|      39|        1|
|            2|            A Lamusi|  male|  23|   170|60.0|         China|           2|     NA|     2|      49|        2|
|            3| Gunnar Nielsen Aaby|  male|  24|     0| 0.0|       Denmark|           3|     NA|     3|       7|        3|
|            4|Edgar Lindenau Aabye|  male|  34|     0| 0.0|Denmark/Sweden|           4|   Gold|     4|       2|        4|
|            5|Christine Jacoba ...|female|  21|   185|82.0|   Netherlands|          10|     NA|     5|      40|        6|
+-------------+-

In [9]:
juegos_df=juegos_df.withColumnRenamed('_c0', 'play_id')
juegos_df.show(8)

+-------+-------------+-----+---------+---------+
|play_id| nombre_juego|annio|temporada|   ciudad|
+-------+-------------+-----+---------+---------+
|      1|  1896 Verano| 1896|   Verano|   Athina|
|      2|  1900 Verano| 1900|   Verano|    Paris|
|      3|  1904 Verano| 1904|   Verano|St. Louis|
|      4|  1906 Verano| 1906|   Verano|   Athina|
|      5|  1908 Verano| 1908|   Verano|   London|
|      6|  1912 Verano| 1912|   Verano|Stockholm|
|      7|  1920 Verano| 1920|   Verano|Antwerpen|
|      8|1924 Invierno| 1924| Invierno| Chamonix|
+-------+-------------+-----+---------+---------+
only showing top 8 rows



23/11/14 08:29:44 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , nombre_juego, annio, temporada, ciudad
 Schema: _c0, nombre_juego, annio, temporada, ciudad
Expected: _c0 but found: 
CSV file: file:///home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/39_Spark_y_Big_Data_Fundamentos/curso_spark_git_clone/curso-apache-spark-platzi/files/juegos.csv


In [10]:
dep_team_year = dep_team_year.join(juegos_df, dep_team_year.juego_id==juegos_df.play_id, 'left')

In [11]:
dep_team_year = dep_team_year.select('deportista_id', 'nombre', 'genero', 'edad', 'altura', 'peso', 'equipo', 'medalla', 'annio', 'temporada', 'ciudad')
dep_team_year.orderBy(f.desc('medalla')).show(8)

23/11/14 08:29:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: , annio, temporada, ciudad
 Schema: _c0, annio, temporada, ciudad
Expected: _c0 but found: 
CSV file: file:///home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/39_Spark_y_Big_Data_Fundamentos/curso_spark_git_clone/curso-apache-spark-platzi/files/juegos.csv


+-------------+--------------------+------+----+------+----+-------------+-------+-----+---------+--------------+
|deportista_id|              nombre|genero|edad|altura|peso|       equipo|medalla|annio|temporada|        ciudad|
+-------------+--------------------+------+----+------+----+-------------+-------+-----+---------+--------------+
|           20|  Kjetil Andr Aamodt|  male|  20|   176|85.0|       Norway| Silver| 1994| Invierno|   Lillehammer|
|        67818|       Lee Chong Wei|  male|  21|   170|68.0|     Malaysia| Silver| 2016|   Verano|Rio de Janeiro|
|           90|Tamila Rashidovna...|female|  21|   163|60.0|       Russia| Silver| 2004|   Verano|        Athina|
|        67865|          Lee EunSil|female|  23|   167|55.0|  South Korea| Silver| 2004|   Verano|        Athina|
|           20|  Kjetil Andr Aamodt|  male|  20|   176|85.0|       Norway| Silver| 1994| Invierno|   Lillehammer|
|        67818|       Lee Chong Wei|  male|  21|   170|68.0|     Malaysia| Silver| 2012|

In [13]:
dep_team_year.groupBy('medalla').count().show()

23/11/14 08:36:37 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 
 Schema: _c0
Expected: _c0 but found: 
CSV file: file:///home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/39_Spark_y_Big_Data_Fundamentos/curso_spark_git_clone/curso-apache-spark-platzi/files/juegos.csv


+-------+------+
|medalla| count|
+-------+------+
|     NA|231333|
| Silver| 13116|
|   Gold| 13372|
| Bronze| 13295|
+-------+------+



In [47]:
dep_team_year.groupBy('nombre', 'equipo', 'medalla')\
    .agg(f.count('*').alias('total_medallas'))\
    .orderBy(f.desc('total_medallas'))\
    .where((dep_team_year.medalla != 'NA') & (dep_team_year.equipo.contains('Argentin')))\
    .show()

23/11/14 09:09:52 WARN CSVHeaderChecker: CSV header does not conform to the schema.
 Header: 
 Schema: _c0
Expected: _c0 but found: 
CSV file: file:///home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/39_Spark_y_Big_Data_Fundamentos/curso_spark_git_clone/curso-apache-spark-platzi/files/juegos.csv


+--------------------+---------+-------+--------------+
|              nombre|   equipo|medalla|total_medallas|
+--------------------+---------+-------+--------------+
|Agustina Soledad ...|Argentina| Bronze|             2|
|Carlos Mauricio E...|Argentina| Silver|             2|
|Luciana Paula Luc...|Argentina| Silver|             2|
| Claudia Ins Burkart|Argentina| Bronze|             2|
|Mara de la Paz Ma...|Argentina| Bronze|             2|
|Santiago Ral Lang...|Argentina| Bronze|             2|
|Carlos Mauricio E...|Argentina| Bronze|             2|
|Mariana Alejandra...|Argentina| Bronze|             2|
|Alejandra Laura G...|Argentina| Bronze|             2|
|Luciana Paula Luc...|Argentina| Bronze|             2|
|Juan Mara de la F...|Argentina| Bronze|             2|
|Mara Magdalena Ai...|Argentina| Bronze|             2|
|Javier Alejandro ...|Argentina|   Gold|             2|
|Mara Mercedes Mec...|Argentina| Bronze|             2|
|         Marin Russo|Argentina| Bronze|        

In [41]:
!pwd 
!ls -lac /home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/09_manipulación_transformacion_de_datos_Pandas_NumPy

/home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/39_Spark_y_Big_Data_Fundamentos
total 1792
drwxr-xr-x  4 compu_dell_ubuntu_01 compu_dell_ubuntu_01   4096 Sep  8 11:11 .
drwxr-xr-x 40 compu_dell_ubuntu_01 compu_dell_ubuntu_01   4096 Nov 12 11:15 ..
drwxr-xr-x  8 compu_dell_ubuntu_01 compu_dell_ubuntu_01   4096 Jun 21 12:15 .git
-rw-r--r--  1 compu_dell_ubuntu_01 compu_dell_ubuntu_01   4662 Jun 13 16:44 .gitignore
drwxr-xr-x  2 compu_dell_ubuntu_01 compu_dell_ubuntu_01   4096 Sep  8 11:10 .ipynb_checkpoints
-rw-r--r--  1 compu_dell_ubuntu_01 compu_dell_ubuntu_01 377574 Sep  8 11:11 09_manipulación_transformacion_de_datos_Pandas_NumPy_notes.ipynb
-rw-r--r--  1 compu_dell_ubuntu_01 compu_dell_ubuntu_01    385 Jun 21 12:12 README.md
-rw-r--r--  1 compu_dell_ubuntu_01 compu_dell_ubuntu_01  51161 Jun 14 17:58 bestsellers-with-categories.csv
-rw-r--r--  1 compu_dell_ubuntu_01 compu_dell_ubuntu_01 377399 Jul 24 00:10 english_notes.ipynb
-rw-r--r--  1 compu_dell_ubuntu_01 c

In [42]:
json_path='/home/compu_dell_ubuntu_01/platzi/pip_y_entornos_virtuales/py-project/09_manipulación_transformacion_de_datos_Pandas_NumPy/'


In [45]:
json_df=spark.read.options(multiline=True)\
    .json(json_path+'hpcharactersdataraw.json')

json_df.printSchema()

root
 |-- Blood: string (nullable = true)
 |-- Descr: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Name: string (nullable = true)
 |-- Profession: string (nullable = true)
 |-- School: string (nullable = true)
 |-- Species/Race: string (nullable = true)



In [49]:
schema=StructType([
    StructField('Name', StringType(), True),
    StructField('Link', StringType(), False),
    StructField('Descr', StringType(), False),
    StructField('Gender', StringType(), False),
    StructField('Species/Race', StringType(), False),
    StructField('Blood', StringType(), False),
    StructField('School', StringType(), False),
    StructField('Profession', StringType(), False)
    
])

In [52]:
json_df_02=spark.read.options(multiline=True).schema(schema)\
    .json(json_path+'hpcharactersdataraw.json')

json_df_02.printSchema()

root
 |-- Name: string (nullable = true)
 |-- Link: string (nullable = true)
 |-- Descr: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Species/Race: string (nullable = true)
 |-- Blood: string (nullable = true)
 |-- School: string (nullable = true)
 |-- Profession: string (nullable = true)



In [58]:
json_df_02.show()

+--------------------+--------------------+--------------------+------+------------+-----------+--------------------+--------------------+
|                Name|                Link|               Descr|Gender|Species/Race|      Blood|              School|          Profession|
+--------------------+--------------------+--------------------+------+------------+-----------+--------------------+--------------------+
|         Mrs. Abbott|https://www.hp-le...|Mrs. Abbott was t...|Female|       Witch|Muggle-born|             Unknown|             Unknown|
|       Hannah Abbott|https://www.hp-le...|Hannah Abbott is ...|Female|       Witch| Half-blood|Hogwarts - Huffle...|Landlady of the L...|
|       Abel Treetops|https://www.hp-le...|Abel Treetops was...|  Male|      Wizard|    Unknown|             Unknown|             Unknown|
|    Euan Abercrombie|https://www.hp-le...|Euan Abercrombie ...|  Male|      Wizard|    Unknown|Hogwarts - Gryffi...|             Unknown|
|Aberforth Dumbledore|https

In [63]:
json_df_02.filter(json_df_02.Gender == 'Female').show()

+------------------+--------------------+--------------------+------+------------+-----------+--------------------+--------------------+
|              Name|                Link|               Descr|Gender|Species/Race|      Blood|              School|          Profession|
+------------------+--------------------+--------------------+------+------------+-----------+--------------------+--------------------+
|       Mrs. Abbott|https://www.hp-le...|Mrs. Abbott was t...|Female|       Witch|Muggle-born|             Unknown|             Unknown|
|     Hannah Abbott|https://www.hp-le...|Hannah Abbott is ...|Female|       Witch| Half-blood|Hogwarts - Huffle...|Landlady of the L...|
|      Agatha Chubb|https://www.hp-le...|Agatha Chubb was ...|Female|       Witch|    Unknown|             Unknown|Expert in ancient...|
|      Agatha Timms|https://www.hp-le...|Agatha Timms was ...|Female|       Witch|    Unknown|             Unknown|          Eel farmer|
|             Agnes|https://www.hp-le...|

In [50]:
new_deportista_df.createOrReplaceTempView('deportista_tv')
deporte_df.createOrReplaceTempView('deporte_tv')
evento_df.createOrReplaceTempView('evento_tv')
juegos_df.createOrReplaceTempView('juegos_tv')
paises_df.createOrReplaceTempView('paises_tv')
resultados_df.createOrReplaceTempView('resultados_tv')

In [53]:
spark.sql('''
SELECT * FROM paises_tv
''').show()

+---+--------------------+-----+
| id|              equipo|sigla|
+---+--------------------+-----+
|  1|         30. Februar|  AUT|
|  2|A North American ...|  MEX|
|  3|           Acipactli|  MEX|
|  4|             Acturus|  ARG|
|  5|         Afghanistan|  AFG|
|  6|            Akatonbo|  IRL|
|  7|            Alain IV|  SUI|
|  8|             Albania|  ALB|
|  9|              Alcaid|  POR|
| 10|            Alcyon-6|  FRA|
| 11|            Alcyon-7|  FRA|
| 12|           Aldebaran|  ITA|
| 13|        Aldebaran II|  ITA|
| 14|              Aletta|  IRL|
| 15|             Algeria|  ALG|
| 16|         Ali-Baba II|  SWE|
| 17|         Ali-Baba IV|  SUI|
| 18|         Ali-Baba IX|  SUI|
| 19|         Ali-Baba VI|  SUI|
| 20|             Allegro|  FRA|
+---+--------------------+-----+
only showing top 20 rows



In [60]:
spark.sql('''
SELECT COUNT(deportista_id), 
    (
    SELECT deportista_id, nombre, edad, altura, peso, equipo FROM deportista_tv d
    LEFT JOIN paises_tv p
    ON d.equipo_id = p.id
    WHERE p.id IS NULL
    ) as TOTAL
FROM TOTAL
''').show()

AnalysisException: [TABLE_OR_VIEW_NOT_FOUND] The table or view `TOTAL` cannot be found. Verify the spelling and correctness of the schema and catalog.
If you did not qualify the name with a schema, verify the current_schema() output, or qualify the name with the correct schema and catalog.
To tolerate the error on drop use DROP VIEW IF EXISTS or DROP TABLE IF EXISTS.; line 9 pos 5;
'Project [unresolvedalias('COUNT('deportista_id), None), scalar-subquery#1979 [] AS TOTAL#1980]
:  +- 'Project ['deportista_id, 'nombre, 'edad, 'altura, 'peso, 'equipo]
:     +- 'Filter isnull('p.id)
:        +- 'Join LeftOuter, ('d.equipo_id = 'p.id)
:           :- 'SubqueryAlias d
:           :  +- 'UnresolvedRelation [deportista_tv], [], false
:           +- 'SubqueryAlias p
:              +- 'UnresolvedRelation [paises_tv], [], false
+- 'UnresolvedRelation [TOTAL], [], false


In [46]:
json_df.show()

+-----------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+
|      Blood|               Descr|Gender|                Link|                Name|          Profession|              School|Species/Race|
+-----------+--------------------+------+--------------------+--------------------+--------------------+--------------------+------------+
|Muggle-born|Mrs. Abbott was t...|Female|https://www.hp-le...|         Mrs. Abbott|             Unknown|             Unknown|       Witch|
| Half-blood|Hannah Abbott is ...|Female|https://www.hp-le...|       Hannah Abbott|Landlady of the L...|Hogwarts - Huffle...|       Witch|
|    Unknown|Abel Treetops was...|  Male|https://www.hp-le...|       Abel Treetops|             Unknown|             Unknown|      Wizard|
|    Unknown|Euan Abercrombie ...|  Male|https://www.hp-le...|    Euan Abercrombie|             Unknown|Hogwarts - Gryffi...|      Wizard|
| Half-blood|Aberforth Dumb

In [ ]:
spark.stop()